# Queue Length Analysis

## Functionalities
- Plot number of connections to each server and thresholds for queueing and packet dropping (max. number of threads and TCP buffer size, respectively).
- Plot queue length of each server.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
  ...
  [node-n]/
    *_service*.tar.gz
    ...
    apigateway*.tar.gz
```
`*_service*.tar.gz` and `apigateway*.tar.gz` tarballs contain RPC log files named `calls.log` and database query log files named `queries.log`.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2021-11-11-16-06-13"

########## CONNECTION
# Window size
WINDOW_IN_MS = 1

########## THRIFT SERVERS
THRIFT_SOMAXCONN = 128
THRIFT_THREADS = 8

########## POSTGRES SERVERS
PG_SOMAXCONN = 128
PG_MAX_CONNECTIONS = 128

## Notebook Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("..")))
from parsers.query_parser import QueryParser
from parsers.rpc_parser import RPCParser
from utils.utils import *

## RPC Log Parsing

In [ ]:
# Build data frame
rpc = pd.concat(
        [pd.DataFrame.from_dict(RPCParser(logfile).parse()) for logfile in get_rpc_logfiles(EXPERIMENT_DIRNAME)],
        ignore_index=True)

In [ ]:
# (Re) Build columns
start_time = get_experiment_start_time(EXPERIMENT_DIRNAME)
rpc["timestamp"] = rpc.apply(lambda r: (r["timestamp"] - start_time).total_seconds(), axis=1)
rpc["window"] = rpc.apply(lambda r: range(int(r["timestamp"] * 1000) // WINDOW_IN_MS,
        int((r["timestamp"] + r["latency"] / 1000) * 1000) // WINDOW_IN_MS + 1), axis=1)
rpc = rpc.explode("window")

In [ ]:
# Get values
servers = sorted(rpc["server"].unique())

## Microservices Queue Length Analysis

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(servers) * 12))
for (i, server) in enumerate(servers):
    df = rpc[(rpc["server"] == server)]
    if MIN_TIME:
        df = df[(df["timestamp"] >= MIN_TIME)]
    if MAX_TIME:
        df = df[(df["timestamp"] <= MAX_TIME)]
    df = df.groupby(["window"])["window"].count()
    df = df.reindex(range(int(df.index.min()), int(df.index.max()) + 1), fill_value=0)
    ax = fig.add_subplot(len(servers), 1, i + 1)
    ax.grid(alpha=0.75)
    ax.set_xlim((df.index.min(), df.index.max()))
    ax.set_ylim((0, df.values.max()))
    ax.axhline(y=THRIFT_THREADS, ls="--", color="blue", linewidth=5)
    ax.axhline(y=THRIFT_THREADS + THRIFT_SOMAXCONN, ls="--", color="red", linewidth=5)
    df.plot(ax=ax, kind="line",
        title="%s - Number of connections (%s-millisecond window)" % (server, WINDOW_IN_MS),
        xlabel="Time (s)", ylabel="Connections (count)", color="black", grid=True,
        xticks=range(int(df.index.min()), int(df.index.max()) + 1, 60 * (1000 // WINDOW_IN_MS)))

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(servers) * 12))
for (i, server) in enumerate(servers):
    df = rpc[(rpc["server"] == server)]
    if MIN_TIME:
        df = df[(df["timestamp"] >= MIN_TIME)]
    if MAX_TIME:
        df = df[(df["timestamp"] <= MAX_TIME)]
    df = df.groupby(["window"])["window"].count()
    df = df.apply(lambda r: max(r - THRIFT_THREADS, 0))
    df = df.reindex(range(int(df.index.min()), int(df.index.max()) + 1), fill_value=0)
    ax = fig.add_subplot(len(servers), 1, i + 1)
    ax.grid(alpha=0.75)
    ax.set_xlim((df.index.min(), df.index.max()))
    ax.set_ylim((0, df.values.max()))
    ax.axhline(y=THRIFT_SOMAXCONN, ls="--", color="red", linewidth=5)
    df.plot(ax=ax, kind="line",
        title="%s - Queue Length (%s-millisecond window)" % (server, WINDOW_IN_MS),
        xlabel="Time (s)", ylabel="Connections (count)", color="black", grid=True,
        xticks=range(int(df.index.min()), int(df.index.max()) + 1, 60 * (1000 // WINDOW_IN_MS)))

## Query Log Parsing

In [ ]:
# Build data frame
query = pd.concat(
        [pd.DataFrame.from_dict(QueryParser(logfile).parse()) for logfile in get_query_logfiles(EXPERIMENT_DIRNAME)],
        ignore_index=True)

In [ ]:
# (Re) Build columns
start_time = get_experiment_start_time(EXPERIMENT_DIRNAME)
query["timestamp"] = query.apply(lambda r: (r["timestamp"] - start_time).total_seconds(), axis=1)
query["window"] = query.apply(lambda r: range(int(r["timestamp"] * 1000) // WINDOW_IN_MS,
        int((r["timestamp"] + r["latency"] / 1000) * 1000) // WINDOW_IN_MS + 1), axis=1)
query = query.explode("window")

In [ ]:
# Get values
dbnames = sorted(query["dbname"].unique())

## Databases Queue Length Analysis

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(servers) * 12))
for (i, dbname) in enumerate(dbnames):
    df = query[(query["dbname"] == dbname)]
    if MIN_TIME:
        df = df[(df["timestamp"] >= MIN_TIME)]
    if MAX_TIME:
        df = df[(df["timestamp"] <= MAX_TIME)]
    df = df.groupby(["window"])["window"].count()
    df = df.reindex(range(int(df.index.min()), int(df.index.max()) + 1), fill_value=0)
    ax = fig.add_subplot(len(servers), 1, i + 1)
    ax.grid(alpha=0.75)
    ax.set_xlim((df.index.min(), df.index.max()))
    ax.set_ylim((0, df.values.max()))
    ax.axhline(y=PG_MAX_CONNECTIONS, ls="--", color="blue", linewidth=5)
    ax.axhline(y=PG_MAX_CONNECTIONS + PG_SOMAXCONN, ls="--", color="red", linewidth=5)
    df.plot(ax=ax, kind="line",
        title="%s - Number of connections (%s-millisecond window)" % (dbname, WINDOW_IN_MS),
        xlabel="Time (s)", ylabel="Connections (count)", color="black", grid=True,
        xticks=range(int(df.index.min()), int(df.index.max()) + 1, 60 * (1000 // WINDOW_IN_MS)))

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(servers) * 12))
for (i, dbname) in enumerate(dbnames):
    df = query[(query["dbname"] == dbname)]
    if MIN_TIME:
        df = df[(df["timestamp"] >= MIN_TIME)]
    if MAX_TIME:
        df = df[(df["timestamp"] <= MAX_TIME)]
    df = df.groupby(["window"])["window"].count()
    df = df.apply(lambda r: max(r - PG_MAX_CONNECTIONS, 0))
    df = df.reindex(range(int(df.index.min()), int(df.index.max()) + 1), fill_value=0)
    ax = fig.add_subplot(len(servers), 1, i + 1)
    ax.grid(alpha=0.75)
    ax.set_xlim((df.index.min(), df.index.max()))
    ax.set_ylim((0, df.values.max()))
    ax.axhline(y=PG_SOMAXCONN, ls="--", color="red", linewidth=5)
    df.plot(ax=ax, kind="line",
        title="%s - Queue Length (%s-millisecond window)" % (dbname, WINDOW_IN_MS),
        xlabel="Time (s)", ylabel="Connections (count)", color="black", grid=True,
        xticks=range(int(df.index.min()), int(df.index.max()) + 1, 60 * (1000 // WINDOW_IN_MS)))